# LangChain Tools 디버깅

새로 만든 LangChain 기반 tool 함수들을 테스트합니다.

## 테스트 대상
1. `LangChainEngine` - LangChain ChatOpenAI 래퍼
2. `tool_talk`, `tool_action`, `tool_item` - @tool 함수들
3. `tool_turn_resolution_v2` - 통합 함수

## 0. 환경 설정

In [ ]:
import sys
import os
import logging
from pathlib import Path

# 프로젝트 루트 추가
root = Path.cwd().parent
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

# 로깅 설정
logging.basicConfig(level=logging.INFO, format="%(levelname)s %(name)s %(message)s")

print(f"프로젝트 루트: {root}")

In [ ]:
# HF_TOKEN 설정 (필수!)
# 방법 1: 직접 입력
# os.environ["HF_TOKEN"] = "your_huggingface_token_here"

# 방법 2: Google Colab userdata 사용
# from google.colab import userdata
# os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

# 환경변수 확인
hf_token = os.environ.get("HF_TOKEN")
if hf_token:
    print(f"✓ HF_TOKEN: {hf_token[:10]}...{hf_token[-4:]}")
else:
    print("✗ HF_TOKEN이 설정되지 않았습니다!")
    print("  위 셀에서 os.environ['HF_TOKEN'] = 'your_token' 을 설정하세요.")

## 1. LangChainEngine 테스트

In [ ]:
from app.llm.langchain_engine import LangChainEngine

# 엔진 초기화
engine = LangChainEngine()

print(f"model: {engine.model}")
print(f"base_url: {engine.base_url}")
print(f"available: {engine.available}")

In [ ]:
# generate() 테스트
prompt = "안녕하세요. 간단히 인사해주세요."
print(f"prompt: {prompt}")

response = engine.generate(prompt, max_tokens=50)
print(f"\nresponse: {response}")

In [ ]:
# bind_tools 테스트
from langchain_core.tools import tool

@tool
def dummy_add(a: int, b: int) -> int:
    """두 숫자를 더합니다."""
    return a + b

llm_with_tools = engine.get_llm_with_tools([dummy_add])
print(f"llm_with_tools: {type(llm_with_tools)}")

# tool call 테스트
from langchain_core.messages import HumanMessage
msg = llm_with_tools.invoke([HumanMessage(content="3과 5를 더해줘")])
print(f"\ntool_calls: {msg.tool_calls}")

## 2. 시나리오 및 WorldState 준비

In [ ]:
from app.loader import ScenarioLoader
from app.schemas import WorldState, NPCState

# 시나리오 로드
base_path = root / "scenarios"
loader = ScenarioLoader(base_path)
scenario_ids = loader.list_scenarios()

print(f"사용 가능한 시나리오: {scenario_ids}")

if scenario_ids:
    scenario_id = scenario_ids[0]
    assets = loader.load(scenario_id)
    print(f"\n로드된 시나리오: {scenario_id}")
    print(f"NPCs: {assets.get_all_npc_ids()}")
    print(f"Items: {assets.get_all_item_ids()}")

In [ ]:
# 테스트용 WorldState 생성
world = WorldState(
    turn=1,
    npcs={
        "family": NPCState(npc_id="family", trust=0, fear=0, suspicion=0),
        "partner": NPCState(npc_id="partner", trust=0, fear=0, suspicion=1),
        "witness": NPCState(npc_id="witness", trust=0, fear=2, suspicion=0),
    },
    inventory=["casefile_brief", "pattern_analyzer"],
    vars={"clue_count": 0, "fabrication_score": 0},
)

print(f"turn: {world.turn}")
print(f"npcs: {list(world.npcs.keys())}")
print(f"inventory: {world.inventory}")
print(f"vars: {world.vars}")

## 3. Tool 함수 개별 테스트

In [ ]:
from app.tools_langchain import (
    set_tool_context,
    tool_talk,
    tool_action,
    tool_item,
    AVAILABLE_TOOLS,
)

# Tool 컨텍스트 설정
set_tool_context(
    world_state=world,
    assets=assets,
    llm_engine=engine,
    memory_llm=None,  # 메모리 LLM 없이 테스트
)

print(f"AVAILABLE_TOOLS: {[t.name for t in AVAILABLE_TOOLS]}")
print("✓ 컨텍스트 설정 완료")

In [ ]:
# tool_talk 테스트
print("=" * 50)
print("tool_talk 테스트")
print("=" * 50)

npc_id = "family"
message = "그날 무슨 일이 있었나요?"

print(f"npc_id: {npc_id}")
print(f"message: {message}")

result = tool_talk.invoke({"npc_id": npc_id, "message": message})

print(f"\nevent_description: {result.get('event_description', [])}")
print(f"state_delta: {result.get('state_delta', {})}")

In [ ]:
# tool_action 테스트
print("=" * 50)
print("tool_action 테스트")
print("=" * 50)

action = "현장 주변을 조사한다"
print(f"action: {action}")

result = tool_action.invoke({"action_description": action})

print(f"\nevent_description: {result.get('event_description', [])}")
print(f"state_delta: {result.get('state_delta', {})}")

In [ ]:
# tool_item 테스트
print("=" * 50)
print("tool_item 테스트")
print("=" * 50)

item_id = "casefile_brief"
print(f"item_id: {item_id}")

result = tool_item.invoke({"item_id": item_id})

print(f"\nevent_description: {result.get('event_description', [])}")
print(f"state_delta: {result.get('state_delta', {})}")

## 4. tool_turn_resolution_v2 통합 테스트

In [ ]:
from app.tools import tool_turn_resolution_v2

# 테스트 케이스
test_cases = [
    ("talk", "목격자에게 '그날 무슨 일이 있었나요?'라고 묻는다"),
    ("action", "현장 주변을 조사한다"),
    ("item", "메모 패드를 사용한다"),
]

for expected_type, user_input in test_cases:
    print("=" * 60)
    print(f"테스트: {expected_type}")
    print(f"입력: {user_input}")
    print("=" * 60)
    
    result = tool_turn_resolution_v2(user_input, world, assets)
    
    print(f"\nevent_description: {result.event_description}")
    print(f"state_delta: {result.state_delta}")
    print()

## 5. 커스텀 테스트

In [ ]:
# 직접 입력 테스트
user_input = "피해자의 동생에게 사건 당일 뭘 했는지 물어본다"

print(f"입력: {user_input}")
print()

result = tool_turn_resolution_v2(user_input, world, assets)

print(f"event_description: {result.event_description}")
print(f"state_delta: {result.state_delta}")

In [ ]:
# Router가 어떤 tool을 선택하는지 직접 확인
from langchain_core.messages import SystemMessage, HumanMessage

npc_ids = assets.get_all_npc_ids()
inventory = world.inventory

router_system = f"""당신은 인터랙티브 노벨 게임의 의도 분류기입니다.
사용자의 입력을 분석하여 적절한 tool을 선택하세요:

- tool_talk: NPC에게 대화하거나 질문하는 경우
- tool_action: 장소 이동, 조사, 관찰 등 일반적인 행동
- tool_item: 아이템을 사용하거나 적용하는 경우

현재 게임 상태:
- NPC 목록: {npc_ids}
- 인벤토리: {inventory}

사용자 입력을 분석하고 적절한 tool을 호출하세요.
"""

# Router LLM 준비
router_llm = engine.get_llm_with_tools(AVAILABLE_TOOLS)

# 테스트
test_inputs = [
    "목격자에게 인사한다",
    "창고를 조사한다",
    "패턴 분석기를 사용한다",
]

for test_input in test_inputs:
    messages = [
        SystemMessage(content=router_system),
        HumanMessage(content=test_input),
    ]
    response = router_llm.invoke(messages)
    
    print(f"입력: {test_input}")
    if response.tool_calls:
        tc = response.tool_calls[0]
        print(f"  → tool: {tc['name']}, args: {tc['args']}")
    else:
        print(f"  → tool_calls 없음 (fallback)")
    print()